In [1]:
import numpy as np
import matplotlib.pyplot as plt
import skimage
from skimage import io
import cv2
from PIL import Image


# Do NOT modifify the function names

def fade_gradually(img):
    processed = img.copy()

    # TODO_B1
    hsv = cv2.cvtColor(processed, cv2.COLOR_RGB2HSV)
    h, s, v = cv2.split(hsv)
    x = np.linspace(1, 0, processed.shape[1])
    x = np.tile(x, (processed.shape[0], 1))
    s = s * x
    s = s.astype(np.uint8)
    hsv = [h, s, v]
    processed = cv2.merge(hsv,3)
    processed = cv2.cvtColor(processed, cv2.COLOR_HSV2RGB)
    return processed


def image_matting(img):
    processed = img.copy()
    
    # TODO_B2
    ziqi = 6
    processed = img.copy()
    new = np.zeros((np.shape(processed)[0],np.shape(processed)[1],4), np.uint8)
    for i in range(np.shape(processed)[0]):
        for j in range(np.shape(processed)[1]):
            if(processed[i][j][0] < ziqi and processed[i][j][1] < ziqi and processed[i][j][2]< ziqi):
                new[i][j] = [255, 255, 255, 0]
            else:
                new[i][j] = [processed[i][j][0],processed[i][j][1],processed[i][j][2],255]
    processed = new
    
    return processed

# You are incouraged to test your program in the main function

def main():
    cat = io.imread("cat.jpg")
    monkey = io.imread("monkey_island.jpg")
    cat_pro = image_matting(cat)
    monkey_pro = fade_gradually(monkey)
    img = Image.fromarray(cat_pro)
    img.show()
    img = Image.fromarray(monkey_pro)
    img.show()

    pass


if __name__ == "__main__":
    main()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import skimage, math
from skimage import io
from PIL import Image

# Do NOT modifify the function names
def bilinear_interpolation(image, y, x):
    h = image.shape[0]
    w = image.shape[1]

    x1 = max(min(math.floor(x), w - 1), 0)
    y1 = max(min(math.floor(y), h - 1), 0)
    x2 = max(min(math.ceil(x), w - 1), 0)
    y2 = max(min(math.ceil(y), h - 1), 0)
    # print(x1,x2,y1,y2)
    a = float(image[y1, x1])
    b = float(image[y2, x1])
    c = float(image[y1, x2])
    d = float(image[y2, x2])
    dx = x - x1
    dy = y - y1

    new_pixel = a * (1 - dx) * (1 - dy)
    new_pixel += b * dy * (1 - dx)
    new_pixel += c * dx * (1 - dy)
    new_pixel += d * dx * dy
    return round(new_pixel)


def my_resize(img, height, width):
    
    # TODO_C1

    new_image = np.zeros((height, width), img.dtype)  
    orig_height = img.shape[0]
    orig_width = img.shape[1]
    # center column and center row
    x_orig_center = (orig_width-1) / 2
    y_orig_center = (orig_height-1) / 2
    # center of resized image
    x_scaled_center = (width-1) / 2
    y_scaled_center = (height-1) / 2
    # scale in both axes
    scale_x = orig_width / width
    scale_y = orig_height / height

    for y in range(height):
        for x in range(width):
            x_ = (x - x_scaled_center) * scale_x + x_orig_center
            y_ = (y - y_scaled_center) * scale_y + y_orig_center
            new_image[y, x] = bilinear_interpolation(img, y_, x_)

    return new_image

def my_rotation(img, angle):
    rads = math.radians(angle)

    # We consider the rotated image to be of the same size as the original
    rot_img = np.uint8(np.zeros(np.shape(img)))

    # Finding the center point of rotated (or original) image.
    height = np.shape(rot_img)[0]
    width  = np.shape(rot_img)[1]

    midx,midy = (width//2, height//2)

    for i in range(np.shape(rot_img)[0]):
        for j in range(np.shape(rot_img)[1]):
            x= (i-midx)*math.cos(rads)+(j-midy)*math.sin(rads)
            y= -(i-midx)*math.sin(rads)+(j-midy)*math.cos(rads)

            x=round(x)+midx 
            y=round(y)+midy 

            if (x>=0 and y>=0 and x<np.shape(img)[0] and  y<np.shape(img)[1]):
                rot_img[i,j] = img[x,y]

    return rot_img 


def my_rotation_RGB(img, angle):

    # TODO_C2
    rads = math.radians(angle)

    # We consider the rotated image to be of the same size as the original
    rot_img = np.uint8(np.zeros(np.shape(img)))

    # Finding the center point of rotated (or original) image.
    height = np.shape(rot_img)[0]
    width  = np.shape(rot_img)[1]

    midx,midy = (width//2, height//2)

    for i in range(np.shape(rot_img)[0]):
        for j in range(np.shape(rot_img)[1]):
            x= (i-midx)*math.cos(rads)+(j-midy)*math.sin(rads)
            y= -(i-midx)*math.sin(rads)+(j-midy)*math.cos(rads)

            x=round(x)+midx 
            y=round(y)+midy 

            if (x>=0 and y>=0 and x<np.shape(img)[0] and  y<np.shape(img)[1]):
                rot_img[i,j,:] = img[x,y,:]

    return rot_img 

    


# You are incouraged to test your program in the main function

def main():
    Lenna_gray = io.imread("Lenna_gray.jpg")
    Lenna = io.imread("Lenna.jpg")

    new_width = 200
    new_height = 400

    resized_img = my_resize(Lenna_gray, new_height, new_width)
    
    img = Image.fromarray(resized_img)
    img.show()
    image_filename = "resized_img.jpg"

    rotated_img = my_rotation(Lenna_gray,90)
    
    img = Image.fromarray(rotated_img)
    img.show()
    image_filename = "rotated_img.jpg"

    rotated_img = my_rotation_RGB(Lenna,90)
    
    img = Image.fromarray(rotated_img)
    img.show()
    image_filename = "rotated_img_RGB.jpg"
    pass


if __name__ == "__main__":
    main()